## **Introducción a SparkSQL**


Este cuaderno cubre las operaciones básicas de Apache SparkSQL.


![](http://spark.apache.org/images/spark-logo.png)


### Objetivos


Spark SQL es un módulo de Spark para el procesamiento de datos estructurados. Se utiliza para consultar datos estructurados dentro de programas de Spark, utilizando SQL o una API de DataFrame familiar.

Después de completar este laboratorio, podrás:

* Cargar un archivo de datos en un DataFrame.
* Crear una vista de tabla para el DataFrame.
* Ejecutar consultas SQL básicas y agregar datos en la vista de la tabla.
* Crear un UDF de Pandas para realizar operaciones a nivel de columna.

### Configuración


Para este cuaderno, utilizaremos Python y Spark (PySpark). Estas bibliotecas deberían estar instaladas en tu entorno local. Pandas es un paquete popular para ciencia de datos en Python. En este cuaderno, usaremos Pandas para cargar un archivo CSV desde el disco a un DataFrame de Pandas en memoria. PySpark es la API de Spark para Python. 

En este cuaderno, usaremos PySpark para inicializar el contexto de Spark.


In [ ]:
#!pip install pyspark
#!pip install findspark
#!pip install pyarrow==0.15.1 
!pip install pandas
!pip install numpy

In [ ]:
import findspark
findspark.init()

In [ ]:
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

#### Ejercicio 1 - Sesión de Spark


Crea e inicializa la sesión de Spark necesaria para cargar los DataFrames y operar sobre ellos.


#### Tarea 1: Creación de la sesión y el contexto de Spark


In [ ]:
# Creando una clase de contexto de Spark
sc = SparkContext()

# Creando una sesión de Spark
spark = SparkSession \
    .builder \
    .appName("Ejemplo básico de DataFrames en Python con Spark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

#### Tarea 2: Inicializar la sesión de Spark

Para trabajar con DataFrames, solo necesitamos verificar que la instancia de la sesión de Spark ha sido creada.

In [ ]:
spark

### Ejercicio 2 - Carga de datos y creación de una vista de tabla


En esta sección, primero leerás un archivo CSV en un DataFrame de Pandas y luego lo convertirás en un DataFrame de Spark.

Pandas es una biblioteca utilizada para la manipulación y el análisis de datos. Ofrece estructuras de datos y operaciones para crear y manipular objetos de Series y DataFrames. Los datos pueden importarse desde diversas fuentes, como matrices de Numpy, diccionarios de Python y archivos CSV. Pandas permite manipular, organizar y mostrar los datos.

Para crear un DataFrame de Spark, cargaremos un DataFrame externo llamado `mtcars`. Este DataFrame incluye 32 observaciones sobre 11 variables:

| Índice | Nombre de columna | Unidades/Descripción |
| :---: | :--- | :--- |
|[, 1] | mpg | Millas por galón |
|[, 2] | cyl | Número de cilindros |
|[, 3] | disp | Cilindrada (pulg. cúbicas) |  
|[, 4] | hp  | Potencia bruta (caballos de fuerza) |
|[, 5] | drat | Relación del eje trasero |
|[, 6] | wt | Peso (lb/1000) |
|[, 7] | qsec | Tiempo de 1/4 de milla |
|[, 8] | vs  | V/S |
|[, 9] | am | Transmisión (0 = automática, 1 = manual) |
|[,10] | gear | Número de marchas hacia adelante |
|[,11] | carb | Número de carburadores |


#### **Tarea 1: Cargar datos en un DataFrame de Pandas**

Pandas tiene una función conveniente para cargar datos CSV desde una URL directamente en un DataFrame de Pandas.


In [ ]:
# Lee el archivo usando la función `read_csv` de Pandas
mtcars = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/mtcars.csv')

In [ ]:
# Previsualiza algunos registros
mtcars.head()

In [ ]:
mtcars.rename( columns={'Unnamed: 0':'name'}, inplace=True )

#### **Tarea 2: Cargar datos en un DataFrame de Spark**


Usamos la función `createDataFrame` para cargar los datos en un DataFrame de Spark.


In [ ]:
sdf = spark.createDataFrame(mtcars) 


Veamos el esquema del DataFrame de Spark cargado:


In [ ]:
sdf.printSchema()

#### **Tarea 3: Renombrar la columna existente "vs" a "versus" y asignar el nuevo DataFrame a la variable "sdf_new"**

La función `withColumnRenamed()` se usa para renombrar los nombres de las columnas existentes. 
 


In [ ]:
sdf_new = sdf.withColumnRenamed("vs", "versus")

La ejecución de la función anterior no modifica el DataFrame original `sdf`, sino que crea un nuevo DataFrame `sdf_new` con la columna renombrada.


#### **Tarea 4: Ver el nuevo DataFrame**


In [ ]:
sdf_new.head(5)

Observa cómo `vs` ahora ha sido renombrado a `versus` en este DataFrame.


#### **Tarea 5: Crear una vista de tabla**
Crear una vista de tabla en Spark SQL es necesario para ejecutar consultas SQL programáticamente en un DataFrame. Una vista es una tabla temporal para ejecutar consultas SQL. Una vista temporal proporciona un alcance local dentro de la sesión actual de Spark. En este ejemplo, creamos una vista temporal usando la función `createTempView()`.


In [ ]:
sdf.createTempView("cars")

### **Ejercicio 3 - Ejecutar consultas SQL y agregar datos**


Una vez que tenemos una vista de tabla, podemos ejecutar consultas similares a las que haríamos en una tabla SQL. Realizaremos operaciones similares a las del notebook de DataFrames. Sin embargo, la diferencia aquí es que utilizamos directamente consultas SQL.


In [ ]:
# Muestra toda la tabla
spark.sql("SELECT * FROM cars").show()

In [ ]:
# Muestra una columna específica
spark.sql("SELECT mpg FROM cars").show(5)

In [ ]:
# Consulta básica de filtrado para determinar qué autos tienen un alto kilometraje y bajo número de cilindros
spark.sql("SELECT * FROM cars WHERE mpg > 20 AND cyl < 6").show(5)

In [ ]:
# Usa el método where para obtener la lista de autos cuyo millaje por galón es menor a 18
sdf.where(sdf['mpg'] < 18).show(3) 

In [ ]:
# Agrega datos y agrupar por cilindros
spark.sql("SELECT count(*), cyl from cars GROUP BY cyl").show()

### **Ejercicio 4 - Crear un UDF de Pandas para aplicar una operación a nivel de columna**
Apache Spark se ha convertido en el estándar de facto para el procesamiento de big data. Para permitir que los científicos de datos aprovechen el valor del big data, Spark agregó una API de Python en la versión 0.7, con soporte para funciones definidas por el usuario (UDF). Estas funciones operan fila por fila y, por lo tanto, sufren una sobrecarga de serialización e invocación. Como resultado, muchos pipelines de datos definen UDFs en Java y Scala y luego las invocan desde Python.

Los Pandas UDFs, construidos sobre Apache Arrow, brindan lo **mejor de ambos mundos**: la capacidad de definir UDFs de alto rendimiento y baja sobrecarga completamente en Python. En este ejemplo simple, crearemos un **Pandas UDF Escalar** para convertir la columna `wt` de unidades imperiales (1000 libras) a unidades métricas (toneladas métricas).

Además, los UDFs pueden registrarse e invocarse en SQL de forma nativa al registrar una función Python regular utilizando el decorador `@pandas_udf()`. Luego, aplicaremos este UDF a nuestra columna `wt`.


#### **Tarea 1: Importar bibliotecas y registrar un UDF**


In [ ]:
# Importa la función Pandas UDF
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [ ]:
@pandas_udf("float")
def convert_wt(s: pd.Series) -> pd.Series:
    # Fórmula para convertir de unidades imperiales a toneladas métricas
    return s * 0.45

spark.udf.register("convert_weight", convert_wt)

#### **Tarea 2: Aplicar el UDF a la vista de tabla**

Ahora podemos aplicar la función `convert_weight` a la columna `wt` en la tabla `cars`. Esto se hace fácilmente usando la consulta SQL que se muestra a continuación. En este ejemplo, mostramos tanto el peso original (en 1000 libras) como el peso convertido (en toneladas métricas).

In [ ]:
spark.sql("SELECT *, wt AS weight_imperial, convert_weight(wt) as weight_metric FROM cars").show()

#### Ejercicio 5 - Combinación de DataFrames basada en una condición específica


#### Tarea 1-Comprendiendo la operación JOIN


In [ ]:
# Define el DataFrame de muestra 1

data = [("A101", "John"), ("A102", "Peter"), ("A103", "Charlie")] 
columns = ["emp_id", "emp_name"]
dataframe_1 = spark.createDataFrame(data, columns)

In [ ]:
# Define el DataFrame de muestra 2

data = [("A101", "John"), ("A102", "Peter"), ("A103", "Charlie")] 
columns = ["emp_id", "emp_name"]
dataframe_1 = spark.createDataFrame(data, columns)

data = [("A101", 3250), ("A102", 6735), ("A103", 8650)] 
columns = ["emp_id", "salary"] 
dataframe_2 = spark.createDataFrame(data, columns) 

In [ ]:
# Crea un nuevo DataFrame "combined_df" realizando un inner join

combined_df = dataframe_1.join(dataframe_2, on="emp_id", how="inner") 

In [ ]:
# Muestra los datos en combined_df como una lista de Row.

combined_df.collect()

#### Tarea 2 - Rellenar los valores faltantes 


In [ ]:
# Define un DataFrame de muestra 1 con algunos valores faltantes

data = [("A101", 1000), ("A102", 2000), ("A103",None)]
columns = ["emp_id", "salary"]
dataframe_1 = spark.createDataFrame(data, columns)


In [ ]:
dataframe_1.head(3)

Verás que se lanza un error porque el DataFrame contiene un valor nulo.


Observa que en el tercer registro del DataFrame "dataframe_1", la columna “salary” contiene un valor nulo ("na"). Se puede rellenar con un valor utilizando la función `fillna()`. 


In [ ]:
# Rellena el valor faltante de salary con un valor específico

filled_df = dataframe_1.fillna({"salary": 3000})

In [ ]:
filled_df.head(3)

### Preguntas prácticas


#### Pregunta 1-Operaciones básicas en SQL


Muestra todas las filas de autos de la marca Mercedez en la vista `cars` que creamos anteriormente. Los autos de Mercedez tienen el prefijo "Merc" en la columna de nombre del auto.

In [ ]:
# Codigo

Haz doble clic **aquí** para obtener una pista.

<!-- La pista está abajo:

La palabra clave `like` en SQL se usa para identificar patrones. 

-->

Haz doble clic **aquí** para ver la solución.

<!-- La respuesta está abajo:

spark.sql("SELECT * FROM cars where name like 'Merc%'").show()

-->


#### Pregunta 2- Funciones definidas por el usuario (UDF)


En este notebook, creamos una UDF para convertir el peso de unidades imperiales a unidades métricas. Ahora, en este ejercicio, por favor crea una UDF en pandas para convertir la columna `mpg` a `kmpl` (kilómetros por litro). Puedes usar el factor de conversión de 0.425.


In [ ]:
# Bloque de código para que los estudiantes respondan
from pyspark.sql.functions import pandas_udf

@pandas_udf("float")
def convert_mileage(s: pd.Series) -> pd.Series:
    # La fórmula para convertir de imperial a unidades métricas
    return s * 0.425

spark.udf.register("convert_mileage", convert_mileage)

spark.sql("SELECT *, mpg AS mpg, convert_mileage(mpg) as kmpl FROM cars").show()

Haz doble clic **aquí** para ver la solución.

<!-- La respuesta está abajo:
from pyspark.sql.functions import pandas_udf

@pandas_udf("float")
def convert_mileage(s: pd.Series) -> pd.Series:
    # La fórmula para convertir de imperial a unidades métricas
    return s * 0.425

spark.udf.register("convert_mileage", convert_mileage)

spark.sql("SELECT *, mpg AS mpg, convert_mileage(mpg) as kmpl FROM cars").show()
-->